In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [9]:
from matplotlib import rcParams 
rcParams.update({'font.size': 18})
from matplotlib.ticker import PercentFormatter

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow_probability as tfp

In [3]:
tf.__version__, tfp.__version__

('2.4.1', '0.12.1')

In [10]:
pickle_in = open("../data/oxygen.pickle", "rb")
oxygen = pickle.load(pickle_in)
oxygen.describe()

,Year,Yday,Hour,PRESSURE,TEMPERATURE_VALUE,SALINITY_VALUE,OXYGEN_VALUE
count,1.234100e+04,1.234100e+04,1.234100e+04,1.234100e+04,1.234100e+04,1.234100e+04,12341.000000
mean,3.848916e-14,-2.301232e-16,4.863017e-15,-1.513703e-16,-1.781251e-16,3.503572e-14,218.501794
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,23.150133
min,-2.035476e+00,-1.754102e+00,-3.602873e-02,-1.421978e+00,-2.515894e+00,-3.751239e+00,109.900000
25%,-7.417802e-01,-8.565878e-01,-3.602873e-02,-7.603727e-01,-7.337237e-01,-6.594490e-01,200.130000
50%,8.148080e-02,3.137826e-02,-3.602873e-02,-9.876695e-02,-1.080223e-01,3.997917e-02,223.360000
75%,7.871331e-01,8.716042e-01,-3.602873e-02,8.936417e-01,6.471345e-01,6.996671e-01,235.430000
max,1.728003e+00,1.721378e+00,2.775338e+01,2.448415e+00,3.421796e+00,2.495926e+00,324.320000


In [11]:
%load_ext tensorboard

In [18]:
# check if there are any GPUs avaialable on Katana
if tf.test.gpu_device_name() != '/device:GPU:0':
  print('WARNING: GPU device not found.')
  strategy = tf.distribute.get_strategy()
else:
  print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))
  strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")

SUCCESS: Found GPU: /device:GPU:0


In [19]:
## Train test split
train_dataset = oxygen.sample(frac=0.8, random_state=0)
test_dataset  = oxygen.drop(train_dataset.index)
train_labels = train_dataset.pop('OXYGEN_VALUE')
test_labels = test_dataset.pop('OXYGEN_VALUE')

In [20]:
def build_model():
    model = keras.Sequential([
      layers.Dense(8, activation='relu', input_shape=[len(train_dataset.keys())]),
      layers.Dense(8, activation='relu'),
      layers.Dense(1)
    ])
    
    optimizer = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss=tf.keras.losses.mean_absolute_percentage_error,
                 optimizer=optimizer,
                 metrics=['mae', 'mse', keras.metrics.MeanAbsolutePercentageError()])
    
    return model

In [26]:
%%timeit -r1 -n1
# build model
strategy = tf.distribute.get_strategy()
with strategy.scope(): model = build_model()

## Train the model
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 10 == 0: print('loss: {:7.4f}, val_loss: {:7.4f}' .format(logs['loss'], logs['val_loss']))
    print('.', end='')
# early stopping callback
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)

EPOCHS = 1000

history = model.fit(
  train_dataset, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot(), es_callback])

loss: 98.5717, val_loss: 95.3647
..........loss:  6.7247, val_loss:  6.8093
..........loss:  5.6207, val_loss:  5.8132
..........loss:  5.2917, val_loss:  5.5342
..........loss:  5.1476, val_loss:  5.4037
..........loss:  5.0792, val_loss:  5.3195
..........loss:  5.0243, val_loss:  5.2801
..........loss:  4.9953, val_loss:  5.2527
..........loss:  4.9619, val_loss:  5.2415
..........loss:  4.9501, val_loss:  5.2374
..........loss:  4.9358, val_loss:  5.2409
..........loss:  4.9245, val_loss:  5.1817
..........loss:  4.9068, val_loss:  5.1568
..........loss:  4.8970, val_loss:  5.1823
..........loss:  4.8951, val_loss:  5.1596
..........loss:  4.8882, val_loss:  5.1365
..........loss:  4.8739, val_loss:  5.1678
..........loss:  4.8770, val_loss:  5.1445
..........loss:  4.8766, val_loss:  5.1484
..........loss:  4.8691, val_loss:  5.1349
..........loss:  4.8587, val_loss:  5.1170
..........loss:  4.8589, val_loss:  5.1186
..........loss:  4.8491, val_loss:  5.1829
..........loss:  4.84

In [29]:
# test execution time with GPU
%%timeit -r1 -n1
# build model
strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
with strategy.scope(): model = build_model()

## Train the model
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 10 == 0: print('loss: {:7.4f}, val_loss: {:7.4f}' .format(logs['loss'], logs['val_loss']))
    print('.', end='')
# early stopping callback
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)

EPOCHS = 1000

history = model.fit(
  train_dataset, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot(), es_callback])

loss: 99.4606, val_loss: 98.0972
..........loss:  7.2363, val_loss:  7.1240
..........loss:  5.6509, val_loss:  5.7970
..........loss:  5.2243, val_loss:  5.4566
..........loss:  5.1091, val_loss:  5.3905
..........loss:  5.0439, val_loss:  5.3260
..........loss:  4.9850, val_loss:  5.2936
..........loss:  4.9444, val_loss:  5.2524
..........loss:  4.9316, val_loss:  5.2203
..........loss:  4.9170, val_loss:  5.2372
..........loss:  4.9079, val_loss:  5.2322
..........loss:  4.9048, val_loss:  5.2151
..........loss:  4.9007, val_loss:  5.1971
..........loss:  4.8937, val_loss:  5.2435
..........loss:  4.8871, val_loss:  5.1959
..........loss:  4.8828, val_loss:  5.1971
..........loss:  4.8785, val_loss:  5.1897
..........loss:  4.8780, val_loss:  5.1710
..........loss:  4.8679, val_loss:  5.1823
..........loss:  4.8623, val_loss:  5.1537
..........loss:  4.8465, val_loss:  5.1723
..........loss:  4.8046, val_loss:  5.1089
..........loss:  4.6494, val_loss:  4.9367
..........loss:  4.51

In [30]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [31]:
def build_epistemic_model():
    train_size = len(train_dataset)
    model = keras.Sequential([
      tfp.layers.DenseVariational(8, activation='relu', make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size, input_shape=[len(train_dataset.keys())]),
      tfp.layers.DenseVariational(8, activation='relu', make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size),
      layers.Dense(1)
    ])
    
    optimizer = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss=tf.keras.losses.mean_absolute_percentage_error,
                 optimizer=optimizer,
                 metrics=['mae', 'mse', keras.metrics.MeanAbsolutePercentageError()])
    
    return model

In [32]:
# test execution time with GPU
%%timeit -r1 -n1
# build model
strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
with strategy.scope(): model = build_epistemic_model()

## Train the model
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 10 == 0: print('loss: {:7.4f}, val_loss: {:7.4f}' .format(logs['loss'], logs['val_loss']))
    print('.', end='')
# early stopping callback
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)

EPOCHS = 500

history = model.fit(
  train_dataset, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot(), es_callback])

Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
loss: 98.7689, val_loss: 97.5924
..........loss: 14.5959, val_loss: 14.1201
..........loss:  8.8195, val_loss:  8.6998
..........loss:  7.5583, val_loss:  7.9421
..........loss:  7.1837, val_loss:  7.5796
..........loss:  7.0360, val_loss:  7.0022
..........loss:  6.8870, val_loss:  7.1840
..........loss:  6.9968, val_loss:  7.1254
..........loss:  6.8267, val_loss:  7.1443
..........loss:  6.8310, val_loss:  7.2421
..........loss:  6.8618, val_loss:  7.0566
..........loss:  6.9348, val_loss:  7.0400
..........loss:  6.8353, val_loss:  7.0084
..........loss:  6.7429, val_loss:  7.0333
..........loss:  6.8074, val_loss:  7.1355
..........loss:  6.6529, val_loss:  6.8315
..........loss:  6.7363, val_loss:  7.0320
..........loss:  6.7291, val_loss:  7.0408
..........loss:  6.5653, val_loss:  6.8965
..........loss:  6.7711, val_loss:  6.7905
..........loss:  6.6273, val_loss:  6.8072
..1 loop, best of 1:

In [33]:
def build_alleatoric_model():
    train_size = len(train_dataset)
    inputs = keras.Input(shape = [len(train_dataset.keys())])
    x = tfp.layers.DenseVariational(8, activation='relu', make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size, input_shape=[len(train_dataset.keys())])(inputs)
    x = tfp.layers.DenseVariational(8, activation='relu', make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size)(x)
    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(x)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)  
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    optimizer = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss=tf.keras.losses.mean_absolute_percentage_error,
                 optimizer=optimizer,
                 metrics=['mae', 'mse', keras.metrics.MeanAbsolutePercentageError()])
    
    return model

In [ ]:
# test execution time with GPU
%%timeit -r1 -n1
# build model
strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
with strategy.scope(): model = build_alleatoric_model()

## Train the model
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 10 == 0: print('loss: {:7.4f}, val_loss: {:7.4f}' .format(logs['loss'], logs['val_loss']))
    print('.', end='')
# early stopping callback
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)

EPOCHS = 500

history = model.fit(
  train_dataset, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot(), es_callback])

loss: 98.9364, val_loss: 97.6537
..........loss: 14.5114, val_loss: 14.1005
..........loss:  8.8985, val_loss:  9.0240
..........loss:  7.5959, val_loss:  7.8728
..........loss:  7.0633, val_loss:  7.4068
..........loss:  7.1069, val_loss:  7.2906
..........loss:  6.9427, val_loss:  7.1031
..........loss:  6.9552, val_loss:  7.1949
..........loss:  6.9351, val_loss:  7.1363
..........loss:  6.8581, val_loss:  7.1123
..........loss:  6.8160, val_loss:  7.1264
.....